# Functional Programming
Suppose you have a world where you don't use classes (gasp!). Why in the world would you want to do that? The reason is that some problems are better suited when you don't mutate an underlying object. Because attributes of an object keeps statefulness of something, there are use cases where state hinders than helps: race conditions in threaded code, debugging complex mutations across multiple method calls, . However, some problems are better where you don't remember something about the instance after it is used--it is stateless. In Functional Programming, it's better to just create a new object.  

If you are a boring, math professor who likes to think that CS is just applied math, then you probably think in terms of the textbook definitions of Functional Programming (often abbreviated as FP): a) __immutability__, b) __pure functions__, and __high-order functions__.  
* __Immutability__: nothing ever changes. What you put into a function, you get something else out. Stay the way you are--never change. Don't update yourself; transform into something new.  
* __Pure functions__: functions ideally should be completely deterministic in that given a specific input, the output will ALWAYS return the same output. In fact, nothing else should change at all; a __side effect__ is something that changes outside the scope of the function. Pure functions, by definition, cannot have a side effect. Sometimes, this constrant is too different, so the lesser constraint is `idempotency`: if something else changes (that is not the output), then something always changes in exactly same way--the side effect is deterministic.  
* __Higher-order functions__: since functions are first-class citizens in Python, what they means is that you can pass functions into functions. You can have a function return a function. You can define functions inside a function.  

But then again, do I <i>look</i> that old? Carpe diem! Let's go through some actual, fun examples of Functional Programming, so we can ditch the textbook answers. Books are for squares! Also, in this notebook we will be tempered in our approach to programming: when to use functional programming and when to mix functional programming with OOP.  


## `map`/`filter`/`reduce`
List comprehensions are generally the "Pythonic" way to do things. Let's try to do the same using FP syntax.  A list comprehension can do both `map` and `filter` and the reverse is true: all list comprehensions are a composition of `map` and `filter`. Instead of updating/modifying a list, just return a new one.

In [1]:
# double element: OOP where mutating/updating a list (using a method)
accumulator = [] # the list is being mutated
for i in range(10):
    accumulator.append(i * 2)
accumulator

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [2]:
# double element: FP equivalent with no mutations
list(map(lambda x: x * 2, range(10)))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [3]:
# remove evens: OOP where mutating/updating a list
accumulator = []
for i in range(10):
    if i % 2:
        accumulator.append(i)
accumulator

[1, 3, 5, 7, 9]

In [4]:
# remove evens: FP equivalent with no mutations
list(filter(lambda x: x % 2, range(10)))

[1, 3, 5, 7, 9]

A list comprehension cannot be used to emulate `reduce`. You'll need a for-loop with an accumulator to get the equivalent of FP's `reduce`. However, the accumulator is stateful (by having to explicitly store a variable) whereas `reduce` does not have any global variable that kept between function calls.  
Note: Guido van Rossum does not like `reduce` and prefers an explicit for-loop. That is why `reduce` is no longer a builtin function in Python 3 and thus has to be imported from `functools`. 

In [5]:
# imperative approach
accumulator = 0
for i in range(10):
    accumulator += i
accumulator

45

In [6]:
# FP: not need for statefulness in the form of an intermediate/temporary accumulator variable
from functools import reduce

reduce(lambda x, y: x + y, range(10))

45

When you are given a hammer, everything looks like a nail. Use Functional Programming when it benefits you. Also, learn when to not force FP into a problem that it is less appropriate. There are some problems that lends itself to OOP and there are some that lend themselves to FP. Let me give you an example where OOP is far superior.   

In [7]:
from string import ascii_letters
from random import choices

print(ascii_letters)
lots_of_letters = choices(ascii_letters, k=100000)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ


In [8]:
%%time
result_OOP = "".join(lots_of_letters) # OOP is much faster since runtime is O(n)

Wall time: 1.99 ms


In [9]:
%%time
result_FP = reduce(lambda x, y: x + y, lots_of_letters) # FP is much slower since runtime is O(n ** 2)

Wall time: 216 ms


In [10]:
result_OOP == result_FP

True

In [7]:
composition, high-order functions
closure is a poor main's object

functional core and imperative shell

high performance frameworks like Apache Spark, Beam, Dask (dataframe, bag, array), Ray
graph based approach: TensorFlow, Airflow, Dask (delayed)

SyntaxError: invalid syntax (<ipython-input-7-bc24622f0476>, line 1)

In fact, you can even stack/chain decorators. The decorator closest to the function the decorator is applied first. Now that I think about it, decorators are in some sense like a generator for functions--decorators change the function call at execution time but are effectively lazy.

In [68]:
def greetings__I_am_close(func):
    def inner(arg):
        print("Hiiiiii! -Kirby")
        return func(arg)
    return inner
    
def farewell__I_am_far(func):
    def inner(arg):
        results = func(arg)
        print("I'll be back! -The Terminator")
        return results
    return inner

@farewell__I_am_far
@greetings__I_am_close
def double(x):
    return x * 2

double(42)

Hiiiiii! -Kirby
I'll be back! -The Terminator


84

Now you have another question, what do you do if you have multiple arguments that you don't know the number of argments or argument names or which arguments are still using the default values. That's where you use `*args` and `**kwargs`! This setup covers all cases.

In [20]:
def identity_decorator(func): # does nothing special
    def inner(*args, **kwargs):
        return func(*args, **kwargs)
    return inner

@identity_decorator
def silly_function(a, b, c=3):
    print(a, b, c)
    
silly_function(1, 2)
silly_function(a=5, b=4)
silly_function(*(6, 7, 8))
silly_function(**{"c": 10, "a": 11, "b": 12})

1 2 3
5 4 3
6 7 8
11 12 10


Now you may ask. What about the docstring? Won't the decorated function contain the decorator's docstring? You are correct. That's why we do functools.wraps() decorator to extract the correct docstring.

In [36]:
def identity_decorator(func):
    """This docstring is totally lost after decorating a function"""
    def inner(*args, **kwargs):
        """Decorator that does nothing special"""
        return func(*args, **kwargs)
    return inner

@identity_decorator
def double(arg):
    "Double the value"
    return arg * 2


help(double)

Help on function inner in module __main__:

inner(*args, **kwargs)
    Decorator that does nothing special



In [37]:
from functools import wraps

def identity_decorator(func):
    """This docstring is totally lost after decorating a function"""
    @wraps(func) # I decorate the inner function, so that I extract the original function's docstring and apply it to the returned inner() function
    def inner(*args, **kwargs):
        """Decorator that does nothing special"""
        return func(*args, **kwargs)
    return inner

@identity_decorator
def double(arg):
    "Double the value"
    return arg * 2


help(double)

Help on function double in module __main__:

double(arg)
    Double the value



Now you may another question: can decorators take any arguments? YES! Remember, decorators themselves are just regular functions. The only difference is that inside of taking arguments that contain data and return processed data, decorators take in functions and return "processed/enhanced/augmented" functions. That is why they are called decorators--they decorate the original function.

For example, a naive implementation of fibonacci has O(n^2) runtime complexity--it has quadratic runtime. 

In [43]:
%%time
import time

def fibonacci_recursion(num):
    print("current number is {}".format(num))
    time.sleep(0.1)
    if (num == 1) or (num == 2):
        return 1
    else:
        return fibonacci_recursion(num - 1) + fibonacci_recursion(num - 2)


fibonacci_recursion(7)

current number is 7
current number is 6
current number is 5
current number is 4
current number is 3
current number is 2
current number is 1
current number is 2
current number is 3
current number is 2
current number is 1
current number is 4
current number is 3
current number is 2
current number is 1
current number is 2
current number is 5
current number is 4
current number is 3
current number is 2
current number is 1
current number is 2
current number is 3
current number is 2
current number is 1
Wall time: 2.52 s


With `lru_cache()` (which stands for Least-Recently-Used cache), we can put fibonacci to O(n) linear runtime. `lru_cache()` has an argument for `maxsize`--how big do you want your cache to be? Decorators that have arguments are basically decorators within decorators. Nested decorators is how they are written: the outer decorator takes configuration(s) as arguments; the inner decorator takes in the target function as the argument.  
Just a side note: caches are useful in a technique called memoization--not a typo, not memorization. Memoization + recursion = dynamic programming.

In [51]:
%%time
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci_recursion(num):
    print("current number is {}".format(num))
    time.sleep(0.1)
    if (num == 1) or (num == 2):
        return 1
    else:
        return fibonacci_recursion(num - 1) + fibonacci_recursion(num - 2)

fibonacci_recursion(7)

current number is 7
current number is 6
current number is 5
current number is 4
current number is 3
current number is 2
current number is 1
Wall time: 705 ms


Now you may have a question. Does decorators work on things outside of functions? YES! Decorators also works on classes or anything that is callable.

In [56]:
import pandas as pd

def polite_class(cls):
    def inner(*args, **kwargs):
        print("Greetings, I come in peace --Never said by the Borg")
        result = cls(*args, **kwargs) # technically you don't need the save it; you can just return
        return result
    return inner

pd.DataFrame = polite_class(pd.DataFrame)
pd.DataFrame({"a": range(5)})

Greetings, I come in peace --Never said by the Borg


,a
0,0
1,1
2,2
3,3
4,4


be aware of mutability: [[1]] * 10 vs [[1] for i in range(10)]. numpy array  

all functions return something  
don't reify generator expression into a list in a function call, especially for reducers  
decorator: are the equivalent of OOP's inheritance: overwrite/extend behavior. Can affect before, after, and during original function call    
functools, starmap  
functools: lru_cache  
import toolz.functoolz  

method chaining/fluent interface vs functional  
currying/partial function?  
closure vs objects  
functoolz/Dask/Pandas .pipe, .assign  

generator vs generator function, chained generator, yield from, coroutine    
infinite generator  
no explicit loops  

functions are first-class citizens  
    Can be used as parameters  
    Can be used as a return value  
    Can be assigned to variables  
    Can be stored in data structures such as hash tables, lists, ...  
    Can have their own attributes 